<a href="https://colab.research.google.com/github/tdiffendal/USAT/blob/master/census-responses/census_responses_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2020 Census Response Rate Analysis

### Theresa Diffendal, USA Today data intern, 06/2020

#### 2020 response rates from: https://2020census.gov/en/response-rates.html
#### 2010 response rates from: https://api.census.gov/data/2010/dec/responserate/variables.html
#### Demographic information in 2014-2018 ACS 5-year-estimate from: https://data2.nhgis.org/main

## Column Names

GEO_ID = Geographic Identifier

RESP_DATE = Posting Date

State = name of state (one of the 50 states, District of Columbia, Puerto Rico, or NaN)

Geo_Name = name of the tract, county, state

Region = region of the U.S. in which state is located as defined by census map at https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf

Geo_Type = type of geography; possible answers include Census Tract, Congressional District, Consolidated City, Country, County, County Subdivision, Place, Region, State, Tribal Tract, Tribal Area

DRRINT = Daily Self-Response Rate - Internet

DRRALL = Daily Self-Response Rate – Overall

CRRINT = Cumulative Self-Response Rate - Internet; renamed internet

not_int = new calculated column showing response rate NOT from internet

CRRALL = Cumulative Self-Response Rate – Overall; renamed 2020_rate

DINTMIN = Minimum Daily Internet Self-Response Rate

DMIN = Minimum Daily Overall Self-Response Rate

CINTMIN = Minimum Cumulative Internet Self-Response Rate

CMIN = Minimum Cumulative Overall Self-Response Rate

DINTMAX = Maximum Daily Internet Self-Response Rate

DMAX = Maximum Daily Overall Self-Response Rate

CINTMAX = Maximum Cumulative Internet Self-Response Rate

CMAX = Maximum Cumulative Overall Self-Response Rate

DINTAVG = Average Daily Internet Self-Response Rate

DAVG = Average Daily Overall Self-Response Rate

CINTAVG = Average Cumulative Internet Self-Response Rate

CAVG = Average Cumulative Overall Self-Response Rate

DINTMED = Median Daily Internet Self-Response Rate

DMED = Median Daily Overall Self-Response Rate

CINTMED = Median Cumulative Internet Self-Response Rate

CMED = Median Cumulative Overall Self-Response Rate

## Read, Merge, Clean Data

### Initial Load and Merge

In [112]:
#mount drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [113]:
import pandas as pd
import numpy as np

# read in 2020 response rates
initial_df = pd.read_csv('https://www2.census.gov/programs-surveys/decennial/2020/data/2020map/2020/decennialrr2020.csv')
# read in as latin-1 since the automatic detection returns an error
crosswalk = pd.read_csv('https://www2.census.gov/programs-surveys/decennial/2020/data/2020map/2020/decennialrr2020_crosswalkfile.csv', encoding='latin-1')
# states paired with region as defined by census map at https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf
regions = pd.read_csv('https://raw.githubusercontent.com/tdiffendal/USAT/master/census-responses/data/state_region.csv')

# merge responses and crosswalk
temp = pd.merge(initial_df, crosswalk, on='GEO_ID')

#merge merged1 with region data
merged = pd.merge(temp, regions, on='State')

# create column showing responses not from internet
merged['not_int'] = merged.CRRALL - merged.CRRINT
merged['not_int_pct'] = (merged.not_int) * 100 / merged.CRRALL

#reorder columns to move State, Geo_Name and Geo_Type to front; also going to drop some values
cols = ['GEO_ID', 'RESP_DATE', 'State', 'Geo_Name', 'Region', 'Geo_Type', 
        'CRRINT', 'not_int', 'not_int_pct', 'CRRALL']
merged = merged[cols].rename(columns={'CRRINT':'internet', 'CRRALL':'2020_rate'})

### States

#### 2020 data

In [114]:
# create df with response rate by state
states2020 = merged[merged['Geo_Type'] == 'State']
states2020 = states2020.rename(columns={"internet": "state_internet", 
                                        "not_int" : "state_not_int", 
                                        'not_int_pct' : 'state_not_int_pct',
                                        "2020_rate" : "2020_state_rate"})

# print df and sort by highest cumulative response rate
#states2020.sort_values(by='2020_state_rate', ascending=False)

#### Join 2010 states

In [115]:
# read in csvs with 2010 response data for states
states2010 = pd.read_csv('https://raw.githubusercontent.com/tdiffendal/USAT/master/census-responses/data/states2010.csv')

# merge with 2020 states
states = pd.merge(states2020, states2010, on='State')
#only select columns we want
cols = ['GEO_ID', 'State', 'Region',
 '2020_state_rate', '2010_rate', '2000_rate']
states = states[cols].rename(
    columns={'2000_rate':'2000_state_rate', '2010_rate':'2010_state_rate'})

#create column with difference in 2010 vs 2020 response rate
states['10_20_state_difference'] = (states['2020_state_rate'] - states['2010_state_rate']) * 100 / states['2010_state_rate']

#print table sorted by 10-20 difference largest ---> smallest
#states.sort_values(by=['2000_state_rate'], ascending=True)

### Census Tracts

#### 2020 Tracts

In [116]:
# select just census tract geo types
tracts2020 = merged[merged['Geo_Type'].str.contains("Tract")].rename(
    columns={"2020_rate": "2020_tract_rate",'not_int':'tract_not_int',
             'not_int_pct':'tract_not_int_pct'})
# sort by highest cumulative response rate
#tracts2020.sort_values(by='2020_tract_rate', ascending=False)

In [117]:
#tract rates compared to state averages
tract2020states = pd.merge(tracts2020, states, on=['State', 'Region'])
tract2020states = tract2020states[['GEO_ID_x', 'State', 'Geo_Name', 'Geo_Type', 
                                   'Region','2020_tract_rate', '2020_state_rate', 
                                   '2010_state_rate', '10_20_state_difference']
                                  ].rename(columns={'GEO_ID_x':'GEO_ID'})
tract2020states['2020_tract_st_diff'] = tract2020states['2020_tract_rate'] - tract2020states['2020_state_rate']
tract2020states.sort_values(by=['2020_tract_st_diff'])

print(
    "Difference in records:", len(tracts2020) - len(tract2020states), "\n",
    "Number of tribal tracts:", len(tracts2020[tracts2020['Geo_Type'].str.contains("Tribal")]),
    "\n\n", "merging tracts with states will drop tribal tracts", "\n",
    "(as they have no state), so those are examined separately below"
)

Difference in records: 426 
 Number of tribal tracts: 426 

 merging tracts with states will drop tribal tracts 
 (as they have no state), so those are examined separately below


#### Join 2010 tract rates

In [118]:
# read in csvs with 2010 response data for tracts and states
tracts2010 = pd.read_csv('https://raw.githubusercontent.com/tdiffendal/USAT/master/census-responses/data/2010responserate.csv'
).rename(columns={'FSRR2010':'2010_tract_rate'})
#tracts2010

In [119]:
## difference in row numbers: both tract dfs have 84519 rows, but when joined only 84093
# Identify what values are in tracts2010 and not in tracts2020
#key_diff1 = set(tracts2010.GEO_ID).difference(tracts2020.GEO_ID)
#len(key_diff1)
#key_diff1

# Identify what values are in tracts2020 and not in tracts2010
#key_diff2 = set(tracts2020.GEO_ID).difference(tracts2010.GEO_ID)
#len(key_diff2)
#key_diff2

# 2010 rates do not include the 426 tribal tracts
# Those differences account for 426 tracts, which is .5% of the original 84519 
# tracts. These tracts are dropped in the comparative analyses and are analyzed separately

In [120]:
# merge with 2020 tracts
tracts = pd.merge(tract2020states, tracts2010, on='GEO_ID')
#select only columns we want
cols = ['Geo_Name','county', 'State_y', 'Region', 'Geo_Type', '2020_tract_rate', 
        '2010_tract_rate', '2020_tract_st_diff', '2020_state_rate', 
        '2010_state_rate', '10_20_state_difference', 'GEO_ID']
tracts = tracts[cols].rename(columns={'State_y':'State'})
#print df sorted largest --> smallest 2010 rate
#tracts.sort_values(by='2010_tract_rate', ascending=False)

In [121]:
#how many null 2010 response values are there
is_temp = tracts.isnull()
no_2010 = is_temp.any(axis=1)
no_2010 = tracts[no_2010].sort_values(by="2010_tract_rate")

#tracts2010[tracts2010['2010_tract_rate'] == 0]

print(
    "There are", len(no_2010), "null 2010 response rate values out of", 
    len(tracts2010), "total 2010 observations, \n or",
    len(no_2010) * 100 / len(tracts2010) , "% \n",
    "and", len(no_2010) * 100 / len(tracts), "% of the", len(tracts),
    "total of 2020 and 2010 tracts \n\n",
    "0 response rate traacts in each state:", "\n",
    no_2010['State'].value_counts())

There are 531 null 2010 response rate values out of 84519 total 2010 observations, 
 or 0.6282611010542009 % 
 and 0.6314437586957298 % of the 84093 total of 2020 and 2010 tracts 

 0 response rate traacts in each state: 
  Wisconsin               92
 Florida                 56
 California              54
 Texas                   54
 Arizona                 50
 New York                50
 New Mexico              30
 Massachusetts           21
 Washington              17
 Montana                 14
 South Dakota            13
 North Carolina           8
 Alabama                  7
 Minnesota                7
 Utah                     6
 Colorado                 6
 Wyoming                  6
 Idaho                    6
 North Dakota             6
 Virginia                 3
 Maine                    3
 Nevada                   3
 New Hampshire            3
 Vermont                  3
 Oklahoma                 2
 Nebraska                 2
 New Jersey               2
 Alaska              

In [122]:
#create column with difference in 2010 vs 2020 response rate
tracts['10_20_tract_difference'] = (tracts['2020_tract_rate'] - tracts['2010_tract_rate']) / tracts['2010_tract_rate']
#sort df largest --> smallest 10-20 difference
#tracts.sort_values(by='10_20_tract_difference', ascending=False)

### Census API

In [159]:
#try something new

import censusdata
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)

ModuleNotFoundError: ignored

### Demographic Data

In [123]:
#load census demographic data, join

## demographics data
temp1 = pd.read_csv('/content/drive/Shared drives/Shared Items/census-responses/data/acs_demographics/ACSDP5Y2018.DP05_data_with_overlays_2020-07-02T144029.csv',
                    usecols = [0,1,2,70,148,152,156,176,208,228,232,284,304,342],
                    names = ['GEO_ID', 'NAME','total_population','median_age', 
                             'white_pct','black_pct','native_pct','asian_pct',
                             'pacific_pct','other_pct','two_pct','latino_pct',
                             'notLatino_pct','house_units'], skiprows=1)

### housing data
temp2 = pd.read_csv('/content/drive/Shared drives/Shared Items/census-responses/data/acs_housing/ACSDP5Y2018.DP04_data_with_overlays_2020-07-02T161352.csv',
                    usecols = [0,1,8,12,28,52,56,184,188,300,354,568],
                    names = ['GEO_ID', 'NAME','occupied_pct','vacant_pct', 
                             'house_pct','bigapt_pct','mobilehome_pct',
                             'owner_pct','renter_pct','no_telephone_pct',
                             'median_value','rent_more_35_pct'], skiprows=2)

## income data
temp3 = pd.read_csv('/content/drive/Shared drives/Shared Items/census-responses/data/acs_income/ACSST5Y2018.S1901_data_with_overlays_2020-07-02T160659.csv',
                    usecols = [0,1,90],
                    names = ['GEO_ID', 'NAME','median_income'], skiprows=2)

## internet access data
temp4 = pd.read_csv('/content/drive/Shared drives/Shared Items/census-responses/data/acs_internet_access/ACSDT5Y2018.B28002_data_with_overlays_2020-07-02T154751.csv',
                    usecols = [0,1,2,26],
                    names = ['GEO_ID', 'NAME','total_comp', 'no_int'], skiprows=2)

#create new column to get % without internet instead of whole number
temp4['no_int_pct'] = temp4.no_int * 100 / temp4.total_comp
cols = ['GEO_ID', 'NAME', 'no_int_pct']
temp4 = temp4[cols]

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,12,28,52,56,184,188,300) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [124]:
## merge demographic and response rate data

frames = [temp1, temp2, temp3, temp4]

# merge to create new df with response rates and demos for all tracts
from functools import reduce
demo = reduce(lambda  left,right: pd.merge(left,right,on=['GEO_ID', 'NAME'],
                                            how='left'), frames)
temp = pd.merge(tracts, demo, on=['GEO_ID'], how = 'left')

temp

,Geo_Name,county,State,Region,Geo_Type,2020_tract_rate,2010_tract_rate,...,owner_pct,renter_pct,no_telephone_pct,median_value,rent_more_35_pct,median_income,no_int_pct
0,"Tract 201, Autauga",Autauga County,Alabama,South,Census Tract,64.9,70.6,...,74.5,25.5,2.9,133300,66.5,58625,26.535948
1,"Tract 202, Autauga",Autauga County,Alabama,South,Census Tract,66.0,70.1,...,64.5,35.5,2.4,93100,44.6,43531,30.458971
2,"Tract 203, Autauga",Autauga County,Alabama,South,Census Tract,74.1,73.6,...,64.9,35.1,0.7,112400,47.7,51875,23.456790
3,"Tract 204, Autauga",Autauga County,Alabama,South,Census Tract,78.1,78.4,...,77.0,23.0,2.5,143100,29.5,54050,17.510677
4,"Tract 205.01, Autauga",Autauga County,Alabama,South,Census Tract,78.3,81.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84088,"Tract 7505.01, Yauco",Yauco Municipio,Puerto Rico,Puerto Rico,Census Tract,34.4,70.4,...,70.6,29.4,3.8,98100,50.0,20179,32.094811
84089,"Tract 7505.02, Yauco",Yauco Municipio,Puerto Rico,Puerto Rico,Census Tract,40.6,73.4,...,67.9,32.1,2.5,129900,49.1,22750,37.323944
84090,"Tract 7505.03, Yauco",Yauco Municipio,Puerto Rico,Puerto Rico,Census Tract,27.7,63.4,...,71.9,28.1,2.2,99600,17.1,18158,33.110368
84091,"Tract 7506.01, Yauco",Yauco Municipio,Puerto Rico,Puerto Rico,Census Tract,28.0,67.1,...,89.9,10.1,2.3,101700,55.6,19332,43.092963


In [125]:
pd.merge(tracts, demo,on='GEO_ID', how='inner').to_csv("/content/drive/My Drive/0USA Today/paired.csv")

#### Cleaning

In [126]:
len(tracts2010)

84519

In [127]:
## difference in row numbers
# Identify what values are in tracts and not in demo
key_diff1 = pd.DataFrame(list(set(tracts.GEO_ID).difference(demo.GEO_ID))
).rename(columns={0:'GEO_ID'})

# Identify what values are in demo and not in tracts
key_diff2 = pd.DataFrame(list(set(demo.GEO_ID).difference(tracts.GEO_ID))
).rename(columns={0:'GEO_ID'})

print("There are", len(tracts), "observations in the tracts df and \n",
    "There are", len(demo), "observations in the demographics df, \n",
    "a difference of", len(tracts) - len(demo), "\n\n",
    "There are", len(key_diff1), 
    "tracts in the tracts df that are not in the demographics df \n",
    "And there are", len(key_diff2), "tracts in the demo df not in the tracts df")

There are 84093 observations in the tracts df and 
 There are 74002 observations in the demographics df, 
 a difference of 10091 

 There are 22307 tracts in the tracts df that are not in the demographics df 
 And there are 12216 tracts in the demo df not in the tracts df


In [128]:
pd.merge(key_diff2, demo, on='GEO_ID', how='left').to_csv("/content/drive/My Drive/0USA Today/demo_unpaired.csv")
pd.merge(key_diff1, tracts, on='GEO_ID', how='left').to_csv("/content/drive/My Drive/0USA Today/tracts_unpaired.csv")
tracts.to_csv("/content/drive/My Drive/0USA Today/tracts.csv")
demo.to_csv("/content/drive/My Drive/0USA Today/demo.csv")

In [129]:
# make a dataframe of all rows with na value
temp1 = temp[temp.isna().any(axis=1)]

# how many nas in each state
temp2 = pd.DataFrame(temp1['State'].value_counts()).reset_index().rename(
    columns={'index':'state', 'State':'count_na'})

#compare to total 
temp3 = pd.DataFrame(temp['State'].value_counts()).reset_index().rename(
    columns={'index':'state', 'State':'count'})

#see what percent of state values of na
temp4 = pd.merge(temp2, temp3, on=['state'])
temp4['na_percent'] = (temp4['count_na']*100) / temp4['count']

print(
    "Total na tracts:", sum(temp4['count_na']), "\n",
    'NAs as % of all tracts:', (sum(temp4['count_na'])*100) / sum(temp4['count']),
    "\n")
len(temp1)
#temp1
#temp1.to_csv("/content/drive/My Drive/0USA Today/nas.csv")

Total na tracts: 22658 
 NAs as % of all tracts: 26.943978690259595 



22658

In [130]:
#check if any nas
#pd.set_option('display.max_rows', None)
#print(pd.isnull(df).sum())

#still 52 states? (50 + DC and PR)
#print("Number states: ", len(df['State'].unique()))

#compare tract numbers now to before na drop
#temp1 = pd.DataFrame(temp.groupby('State')['Geo_Name'].nunique()).rename(columns={'Geo_Name':'beforeDrop'})
#temp2 = pd.DataFrame(df.groupby('State')['Geo_Name'].nunique()).rename(columns={'Geo_Name':'afterDrop'})
#temp3 = pd.merge(temp1, temp2, left_index = True, right_index=True)
#number tracts dropped from each state
#temp3['numDrop'] = temp3['beforeDrop'] - temp3['afterDrop']
# the percentage of total tracts dropped
#temp3['dropPct'] = temp3['numDrop']*100 / temp3['beforeDrop']
#temp3 #puerto rico loses the most tracts at 5.7%

In [131]:
#return column size
pd.set_option('display.max_columns', 15)
#pd.set_option('display.max_row', 50)

#only select columns we want
cols = ['Geo_Name', 'county', 'State', '2020_tract_rate',
 '2010_tract_rate', '2020_tract_st_diff', '2020_state_rate', '2010_state_rate',
 '10_20_state_difference',  '10_20_tract_difference', 'total_population',
 'median_age', 'white_pct', 'black_pct', 'native_pct', 'asian_pct',
 'pacific_pct', 'other_pct', 'two_pct', 'latino_pct', 'notLatino_pct',
 'house_units', 'occupied_pct', 'vacant_pct', 'house_pct', 'bigapt_pct',
 'mobilehome_pct', 'owner_pct', 'renter_pct', 'no_telephone_pct',
 'median_value', 'rent_more_35_pct', 'median_income', 'no_int_pct', 'GEO_ID', 
 'Region']

df = temp[cols]
#print df
#df.to_csv("/gdrive/My Drive/0USA Today/all_rates_demos_merged.csv")
df

,Geo_Name,county,State,2020_tract_rate,2010_tract_rate,2020_tract_st_diff,2020_state_rate,...,no_telephone_pct,median_value,rent_more_35_pct,median_income,no_int_pct,GEO_ID,Region
0,"Tract 201, Autauga",Autauga County,Alabama,64.9,70.6,5.3,59.6,...,2.9,133300,66.5,58625,26.535948,1400000US01001020100,South
1,"Tract 202, Autauga",Autauga County,Alabama,66.0,70.1,6.4,59.6,...,2.4,93100,44.6,43531,30.458971,1400000US01001020200,South
2,"Tract 203, Autauga",Autauga County,Alabama,74.1,73.6,14.5,59.6,...,0.7,112400,47.7,51875,23.456790,1400000US01001020300,South
3,"Tract 204, Autauga",Autauga County,Alabama,78.1,78.4,18.5,59.6,...,2.5,143100,29.5,54050,17.510677,1400000US01001020400,South
4,"Tract 205.01, Autauga",Autauga County,Alabama,78.3,81.2,18.7,59.6,...,NaN,NaN,NaN,NaN,NaN,1400000US01001020501,South
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84088,"Tract 7505.01, Yauco",Yauco Municipio,Puerto Rico,34.4,70.4,10.6,23.8,...,3.8,98100,50.0,20179,32.094811,1400000US72153750501,Puerto Rico
84089,"Tract 7505.02, Yauco",Yauco Municipio,Puerto Rico,40.6,73.4,16.8,23.8,...,2.5,129900,49.1,22750,37.323944,1400000US72153750502,Puerto Rico
84090,"Tract 7505.03, Yauco",Yauco Municipio,Puerto Rico,27.7,63.4,3.9,23.8,...,2.2,99600,17.1,18158,33.110368,1400000US72153750503,Puerto Rico
84091,"Tract 7506.01, Yauco",Yauco Municipio,Puerto Rico,28.0,67.1,4.2,23.8,...,2.3,101700,55.6,19332,43.092963,1400000US72153750601,Puerto Rico


## Analysis

In [132]:
##see all dfs in memory
%whos DataFrame

#### Existing DFs:

#dataframe with all years, states, tracts, demographics
#df

#2010 and 2020 states
#states

#2010 States
#states2010

#2020 States
#states2020

#2010 and 2020 tracts and states
#tracts

#2020 tracts paired with states, includes int data
#tracts2020states

# ignore all dfs with "temp" in name 

Variable          Type         Data/Info
----------------------------------------
big_drop          DataFrame                             <...>n[1000 rows x 13 columns]
crosswalk         DataFrame                   GEO_ID    <...>[125338 rows x 4 columns]
demo              DataFrame                         GEO_<...>[74002 rows x 26 columns]
df                DataFrame                        Geo_N<...>[84093 rows x 36 columns]
initial_df        DataFrame                   GEO_ID   R<...>123252 rows x 22 columns]
inner             DataFrame                       Geo_Na<...>[61786 rows x 38 columns]
is_temp           DataFrame           Geo_Name  county  <...>[84093 rows x 12 columns]
key_diff1         DataFrame                         GEO_<...>n[22307 rows x 1 columns]
key_diff2         DataFrame                         GEO_<...>n[12216 rows x 1 columns]
lowest            DataFrame                             <...>n[1000 rows x 13 columns]
merged            DataFrame                     

In [133]:
# get average state rate and see how many are above average

print(
    "62.0% is the current nationwide response rate and",
    np.sum(states2020['2020_state_rate'] > 62.0),
    "states exceed that", "\n")
print(states2020[states2020['2020_state_rate'] > 62.0].State.to_list(), "\n")

temp = states2020['2020_state_rate'] > 62.0

#how many in each region?
states2020[temp].Region.value_counts()

62.0% is the current nationwide response rate and 23 states exceed that 

['California', 'Colorado', 'Connecticut', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Nebraska', 'New Jersey', 'Ohio', 'Oregon', 'Pennsylvania', 'South Dakota', 'Utah', 'Virginia', 'Washington', 'Wisconsin'] 



Midwest      11
West          6
Northeast     3
South         3
Name: Region, dtype: int64

In [134]:
#how many tracts are > than state avg?

#tract2020States.count(tract2020States['2020_tract_st_diff'] > 0)
print(np.sum(tract2020states['2020_tract_st_diff'] > 0), "tracts out of", 
      len(tract2020states), 
      "total (", 
      (np.sum(tract2020states['2020_tract_st_diff'] > 0) * 100) / len(tract2020states),
      "% ) \n",
      "currently have greater census response rates than their state average")

44939 tracts out of 84093 total ( 53.439644203441425 % ) 
 currently have greater census response rates than their state average


In [135]:
# get info on 1000 tracks with greatest drop in response rate
big_drop = tracts.sort_values(by='10_20_tract_difference', ascending=True).head(1000)
big_drop

,Geo_Name,county,State,Region,Geo_Type,2020_tract_rate,2010_tract_rate,2020_tract_st_diff,2020_state_rate,2010_state_rate,10_20_state_difference,GEO_ID,10_20_tract_difference
52358,"Tract 1.04, Queens",Queens County,New York,Northeast,Census Tract,0.0,25.9,-57.5,57.5,69,-16.666667,1400000US36081000104,-1.000000
61063,"Tract 9800.03, Oklahoma",Oklahoma County,Oklahoma,South,Census Tract,0.0,66.7,-56.3,56.3,68,-17.205882,1400000US40109980003,-1.000000
16024,"Tract 7, Bay",Bay County,Florida,South,Census Tract,0.0,62.2,-59.1,59.1,74,-20.135135,1400000US12005000700,-1.000000
6554,"Tract 2774, Los Angeles",Los Angeles County,California,West,Census Tract,0.0,45.9,-63.1,63.1,73,-13.561644,1400000US06037277400,-1.000000
11899,"Tract 9801, Santa Barbara",Santa Barbara County,California,West,Census Tract,0.0,7.7,-63.1,63.1,73,-13.561644,1400000US06083980100,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10267,"Tract 91.20, San Bernardino",San Bernardino County,California,West,Census Tract,26.0,56.9,-37.1,63.1,73,-13.561644,1400000US06071009120,-0.543058
82801,"Tract 9505.02, Washburn",Washburn County,Wisconsin,Midwest,Census Tract,45.7,100.0,-23.2,68.9,82,-15.975610,1400000US55129950502,-0.543000
83918,"Tract 80.02, San Juan",San Juan Municipio,Puerto Rico,Puerto Rico,Census Tract,25.4,55.5,1.6,23.8,54,-55.925926,1400000US72127008002,-0.542342
44464,"Tract 9403.07, Lake",Lake County,Montana,West,Census Tract,45.8,100.0,-9.9,55.7,68,-18.088235,1400000US30047940307,-0.542000


In [136]:
#lowest tract rates
lowest = tracts.sort_values(by='2020_tract_rate', ascending=True).head(1000)
lowest

,Geo_Name,county,State,Region,Geo_Type,2020_tract_rate,2010_tract_rate,2020_tract_st_diff,2020_state_rate,2010_state_rate,10_20_state_difference,GEO_ID,10_20_tract_difference
33959,"Tract 9517.03, West Feliciana",West Feliciana Parish,Louisiana,South,Census Tract,0.0,0.0,-56.1,56.1,65,-13.692308,1400000US22125951703,NaN
50261,"Tract 53.03, Kings",Kings County,New York,Northeast,Census Tract,0.0,NaN,-57.5,57.5,69,-16.666667,1400000US36047005303,NaN
38744,"Tract 9821, Macomb",Macomb County,Michigan,Midwest,Census Tract,0.0,0.0,-68.2,68.2,78,-12.564103,1400000US26099982100,NaN
15534,"Tract 9800, Kent",Kent County,Delaware,South,Census Tract,0.0,NaN,-59.7,59.7,72,-17.083333,1400000US10001980000,NaN
7979,"Tract 9800.20, Los Angeles",Los Angeles County,California,West,Census Tract,0.0,NaN,-63.1,63.1,73,-13.561644,1400000US06037980020,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38779,"Tract 29.02, Marquette",Marquette County,Michigan,Midwest,Census Tract,21.4,35.0,-46.8,68.2,78,-12.564103,1400000US26103002902,-0.388571
39765,"Tract 5081, Wayne",Wayne County,Michigan,Midwest,Census Tract,21.4,42.0,-46.8,68.2,78,-12.564103,1400000US26163508100,-0.490476
67566,"Tract 9401, Lyman",Lyman County,South Dakota,Midwest,Census Tract,21.4,0.0,-41.8,63.2,76,-16.842105,1400000US46085940100,inf
81291,"Tract 102.04, Monongalia",Monongalia County,West Virginia,South,Census Tract,21.4,58.6,-32.1,53.5,65,-17.692308,1400000US54061010204,-0.634812


### National Comparative Rankings 2010 vs 2020

In [137]:
#Discrepancies? Shouldn't these all match up?
# Average is sum / #obs
#print(
#  "2010 State Resonse Average from states data:",
#    states.mean(axis=0)['2010_state_rate'], "\n",
#  "2010 States Response Average from df data:",
#    df.mean(axis=0)['2010_state_rate'], "\n",
#  "2010 Tract Response Average from df data:",
#    df.mean(axis=0)['2010_tract_rate'],  "\n"
#)
## totals intended to be sum(percentages) / 100 (denominator)
#print(
#  "2010 State Resonse Total from states data:",
#    (states['2010_state_rate'].sum()) / 100, "\n",
#  "2010 States Response Total from df data:",
#    (df['2010_state_rate'].sum()) / 100, "\n",
#  "2010 Tract Response Total from df data:",
#    (df['2010_tract_rate'].sum()) / 100
#)

In [138]:
# Average difference between current state rates and 2010 state rates as of 6/15/20
#takes a while to run so commented out
#df.mean(axis=0)['10_20_state_difference']

#average 2020 response rate across states?
#print(
#  "2020 State Resonse Average from states data:",
#    states.mean(axis=0)['2020_state_rate'], "\n",
#  "2020 States Response Average from df data:",
#   df.mean(axis=0)['2020_state_rate'], "\n",
#  "2020 Tract Response Average from df data:",
#    df.mean(axis=0)['2020_tract_rate'], "\n"
#)

#is the total not sum(numerator) / 100? where numerator is percentage response rate?
#print(
#  "2020 State Resonse Total from states data:",
#    (states['2020_state_rate'].sum()) / (100), "\n",
#  "2020 States Response Total from df data:",
#    (df['2020_state_rate'].sum()) / (100), "\n",
# "2020 Tract Response Total from df data:",
#    (df['2020_tract_rate'].sum()) / (100)
#)

In [139]:
# average difference by region
states.groupby('Region').mean().sort_values(by='10_20_state_difference', ascending=False)

,2020_state_rate,2010_state_rate,2000_state_rate,10_20_state_difference
Region,,,,
Midwest,66.200000,77.307692,79.230769,-14.382402
West,60.084615,70.923077,72.538462,-15.457654
Northeast,60.337500,72.375000,72.875000,-16.721696
South,59.100000,72.176471,70.647059,-18.090618
Puerto Rico,23.800000,54.000000,54.000000,-55.925926


In [140]:
#assign ranks to states based on comparative response rate
states['2020_rank'] = states['2020_state_rate'].rank(method='max', ascending=False)
states['2010_rank'] = states['2010_state_rate'].rank(method='max', ascending=False)

#pull ranks into separate dataframe
state_ranks = states[['State', '2020_rank', '2010_rank']].sort_values(by='2020_rank')

#show change in rank from 2010 to 2020
#negative number means a state has a lower 2020 response rate and has gone down in rankings
state_ranks['rank_change'] = state_ranks['2010_rank'] - state_ranks['2020_rank']
#state_ranks.sort_values(by='rank_change', ascending=True)

#see how many states only changed 2 or fewer positions
#small_change = state_ranks[state_ranks.rank_change.between(-2, 2, inclusive=True)].sort_values(by='rank_change')
#small_change
#16 states have stayed ~similar in the rankings, and this seems to impact
#states with both high and low response rates
#small_change.mean(axis=0)['2020_rank']

### Internet Usage

Internet usage is only available for 2020 rates

In [141]:
### Percent of response rate not from internet

states2020.sort_values(by='state_not_int_pct', ascending=False)
print(
    "The average state response rate to the census NOT conducted online:",
    states2020.mean(axis=0)['state_not_int_pct'], "/n",
    states2020.mean(axis=0)['state_not_int']
)

The average state response rate to the census NOT conducted online: 21.67060402902774 /n 12.803846153846155


In [142]:
# average non internet response rate
states2020.groupby(by='State').mean().sort_values(by='state_internet', ascending=False)

,state_internet,state_not_int,state_not_int_pct,2020_state_rate
State,,,,
Utah,60.4,5.9,8.898944,66.3
Minnesota,60.0,11.5,16.083916,71.5
Washington,58.3,9.0,13.372957,67.3
Wisconsin,56.6,12.3,17.851959,68.9
Maryland,56.3,9.6,14.567527,65.9
Colorado,56.2,9.1,13.935681,65.3
Virginia,54.7,12.0,17.991004,66.7
Illinois,54.5,12.2,18.290855,66.7
Idaho,54.5,10.9,16.666667,65.4


In [143]:
# highest non-internet response rate (not_int)
states2020.sort_values(by='state_not_int', ascending=False)

,GEO_ID,RESP_DATE,State,Geo_Name,Region,Geo_Type,state_internet,state_not_int,state_not_int_pct,2020_state_rate
60537,0400000US28,2020-07-09,Mississippi,Mississippi,South,State,34.5,22.5,39.473684,57.0
4788,0400000US05,2020-07-09,Arkansas,Arkansas,South,State,37.7,19.0,33.509700,56.7
117679,0400000US54,2020-07-09,West Virginia,West Virginia,South,State,35.0,18.5,34.579439,53.5
746,0400000US01,2020-07-09,Alabama,Alabama,South,State,41.4,18.2,30.536913,59.6
43476,0400000US21,2020-07-09,Kentucky,Kentucky,South,State,48.2,17.1,26.186830,65.3
45317,0400000US22,2020-07-09,Louisiana,Louisiana,South,State,40.2,15.9,28.342246,56.1
101175,0400000US47,2020-07-09,Tennessee,Tennessee,South,State,45.5,15.9,25.895765,61.4
35465,0400000US18,2020-07-09,Indiana,Indiana,Midwest,State,50.8,15.5,23.378582,66.3
83303,0400000US39,2020-07-09,Ohio,Ohio,Midwest,State,51.3,15.2,22.857143,66.5
88791,0400000US40,2020-07-09,Oklahoma,Oklahoma,South,State,41.3,15.0,26.642984,56.3


In [144]:
states2020.mean(axis=0)['state_not_int']

12.803846153846155

In [145]:
states2020.mean(axis=0)['state_internet']

47.82884615384617

In [146]:
states2020.mean(axis=0)['2020_state_rate']

60.632692307692324

In [147]:
### Without Puerto Rico

#make non-pr df
no_pr = states2020[states2020['State'] != 'Puerto Rico']

# average internet response
no_pr.mean(axis=0)['state_internet']

48.533333333333346

In [148]:
# average overall response rate
no_pr.mean(axis=0)['2020_state_rate']

61.35490196078433

### Region Analysis

In [149]:
# average by region
# NOTE as tribal tracts are not assigned to a state they do not have a corresponding region and thus are not counted in the regional calculations
states.groupby('Region').mean().sort_values(by='2020_state_rate', ascending=False)

,2020_state_rate,2010_state_rate,2000_state_rate,10_20_state_difference,2020_rank,2010_rank
Region,,,,,,
Midwest,66.200000,77.307692,79.230769,-14.382402,11.538462,13.230769
Northeast,60.337500,72.375000,72.875000,-16.721696,30.250000,31.250000
West,60.084615,70.923077,72.538462,-15.457654,29.153846,35.846154
South,59.100000,72.176471,70.647059,-18.090618,33.235294,31.588235
Puerto Rico,23.800000,54.000000,54.000000,-55.925926,52.000000,52.000000


In [150]:
# average difference as of 6/15/20
#this can take a while to run so is commented out unless needed
#tracts.mean(axis=0)['10_20_tract_difference']

In [151]:
# average difference by region
tracts.groupby('Region').mean().sort_values(by='10_20_tract_difference', ascending=False)

,2020_tract_rate,2010_tract_rate,2020_tract_st_diff,2020_state_rate,2010_state_rate,10_20_state_difference,10_20_tract_difference
Region,,,,,,,
Midwest,66.177469,70.971303,-0.552874,66.730343,77.585668,-13.993341,inf
Northeast,60.785550,67.728176,-0.379024,61.164574,72.578543,-15.778369,inf
South,58.924631,66.022076,-0.157973,59.082604,72.829057,-18.860140,inf
West,62.653973,67.693454,0.038993,62.614979,72.496898,-13.686019,inf
Puerto Rico,23.908686,56.631705,0.108686,23.800000,54.000000,-55.925926,-0.583308


In [152]:
#tract average differences vs state rates
tracts.groupby('State').mean().sort_values(by='2020_state_rate', ascending=False)

,2020_tract_rate,2010_tract_rate,2020_tract_st_diff,2020_state_rate,2010_state_rate,10_20_state_difference,10_20_tract_difference
State,,,,,,,
Minnesota,71.830507,75.075319,0.330507,71.5,81,-11.728395,inf
Wisconsin,68.618122,74.093990,-0.281878,68.9,82,-15.975610,inf
Michigan,68.129174,69.428206,-0.070826,68.2,78,-12.564103,inf
Iowa,67.476682,74.507287,-0.523318,68.0,79,-13.924051,-0.096289
Nebraska,67.088889,72.229068,-0.911111,68.0,77,-11.688312,inf
Washington,66.981787,68.342874,-0.318213,67.3,76,-11.447368,-0.017594
Illinois,65.711797,71.106636,-0.988203,66.7,76,-12.236842,-0.080768
Virginia,66.778634,70.472093,0.078634,66.7,78,-14.487179,-0.049369
Ohio,65.537496,69.750302,-0.962504,66.5,78,-14.743590,-0.066305


### Tribal tracts

In [153]:
# create df with response rates in tribal tracts
tribal = tracts2020[tracts2020['Geo_Type'].str.contains("Tribal")].sort_values(
    by='2020_tract_rate', ascending=False)

In [154]:
### tribal areas and tracts stats

#mean non internet response
tribal.mean(axis=0)['tract_not_int'] #8.37%

10.412676056338025

In [155]:
# mean internet response rate
tribal.mean(axis=0)['internet']

21.48849765258214

In [156]:
# mean overall response rate
tribal.mean(axis=0)['2020_tract_rate']

31.90117370892019

### Tracts with 0 overall response rate

In [157]:
## Tracts with 0 cumulative response rate
is_zero = df['2020_tract_rate'] == 0
zeros = df[is_zero]
zeros.sort_values(by='State')

print(
    "Number of tracts with 0 cumulative response rate:", len(zeros), "\n"
    )

#make dataframe of states with # tracts with 0%, number total tracts, and what % of total tracts are 0
temp = pd.merge(pd.DataFrame(zeros['State'].value_counts()),
                pd.DataFrame(tracts['State'].value_counts()),
                right_index=True, left_index=True).rename(
                    columns={"State_x": "0_tracts", "State_y" : "total_tracts"})
#compute percentage
temp['0_percent'] = temp['0_tracts'] * 100 / temp['total_tracts']
temp

zeros.sort_values(by='State')

Number of tracts with 0 cumulative response rate: 16 



,Geo_Name,county,State,2020_tract_rate,2010_tract_rate,2020_tract_st_diff,2020_state_rate,...,no_telephone_pct,median_value,rent_more_35_pct,median_income,no_int_pct,GEO_ID,Region
1678,"Tract 9800, Coconino",Coconino County,Arizona,0.0,0.0,-58.6,58.6,...,NaN,NaN,NaN,NaN,NaN,1400000US04005980000,West
6554,"Tract 2774, Los Angeles",Los Angeles County,California,0.0,45.9,-63.1,63.1,...,1.9,-,49.2,41103,25.695931,1400000US06037277400,West
7979,"Tract 9800.20, Los Angeles",Los Angeles County,California,0.0,NaN,-63.1,63.1,...,-,-,-,-,NaN,1400000US06037980020,West
11899,"Tract 9801, Santa Barbara",Santa Barbara County,California,0.0,7.7,-63.1,63.1,...,-,-,-,-,NaN,1400000US06083980100,West
15534,"Tract 9800, Kent",Kent County,Delaware,0.0,NaN,-59.7,59.7,...,NaN,NaN,NaN,NaN,NaN,1400000US10001980000,South
16024,"Tract 7, Bay",Bay County,Florida,0.0,62.2,-59.1,59.1,...,5.6,-,35.8,64875,2.716049,1400000US12005000700,South
33959,"Tract 9517.03, West Feliciana",West Feliciana Parish,Louisiana,0.0,0.0,-56.1,56.1,...,NaN,NaN,NaN,NaN,NaN,1400000US22125951703,South
38379,"Tract 9801, Keweenaw",Keweenaw County,Michigan,0.0,0.0,-68.2,68.2,...,-,-,-,-,NaN,1400000US26083980100,Midwest
38744,"Tract 9821, Macomb",Macomb County,Michigan,0.0,0.0,-68.2,68.2,...,-,-,-,-,NaN,1400000US26099982100,Midwest
50261,"Tract 53.03, Kings",Kings County,New York,0.0,NaN,-57.5,57.5,...,NaN,NaN,NaN,NaN,NaN,1400000US36047005303,Northeast


In [158]:
zeros.mean('black_pct')

ValueError: ignored

## Regressions

### With Puerto Rico

#### 2020 Regressions

In [ ]:
#fill in na values with column mean for regression
df = df.dropna(axis=0, how='any')

In [ ]:
variables20

In [ ]:
### 2020 Multi-regression

import statsmodels.api as sm

#put all variables for predicting 2020 rates in dataframe
variables20 = df[['2010_tract_rate', '2010_state_rate',
 'total_population', 'median_age', 'white_pct', 'black_pct', 'native_pct', 
 'asian_pct', 'pacific_pct', 'other_pct', 'two_pct', 'latino_pct', 
 'notLatino_pct', 'house_units', 'occupied_pct', 'vacant_pct', 'house_pct', 
 'bigapt_pct', 'mobilehome_pct', 'owner_pct', 'renter_pct', 'no_telephone_pct',
 'median_value', 'rent_more_35_pct', 'median_income', 'no_int_pct']].astype(float)

#what we want to predict - 2020 response rates - in dataframe
target20 = df[["2020_tract_rate"]].astype(float)

#build model and print summary
model20 = sm.OLS(target20, variables20).fit()
print(model20.summary())

In [ ]:
### 2020 linear regressions for each variable

from sklearn import linear_model

#create list of variable names
cols = variables20.columns.tolist()
#build linear model
regr = linear_model.LinearRegression()
#create empty list to store loop results
rows = []
#loop through each variable
for i in cols:
    #fit linear model to variable
    regr.fit(variables20[[i]], target20)
    #save model variable name, intercept, coef and r^2 to list
    rows.append([i, regr.intercept_, regr.coef_, regr.score(variables20[[i]], target20)])

#turn list into df with these column names
linears20 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets lol
linears20['coefficient'] = linears20['coefficient'].str.get(0)
linears20['coefficient'] = linears20['coefficient'].str.get(0)
linears20['intercept'] = linears20['intercept'].str.get(0)

#print df sorted coefs largest --> smallest
linears20.sort_values(by='coefficient', ascending=False)

#### Normalized 2020 inputs

In [ ]:
### 2020 Multi regression with normalized variables

#normalize variable values
norm_variables20 = (variables20 - variables20.min()) / (variables20.max() - variables20.min())

#build normalized multi-regress model and print summary
norm_model20 = sm.OLS(target20, norm_variables20).fit()
print(norm_model20.summary())

In [ ]:
#2020 normalized linear regressions for each variable

# linear regression for each variable 'i'
cols = norm_variables20.columns.tolist()
# create model
norm_regr = linear_model.LinearRegression()
#empty list for loop results
rows = []
#loop through each variable
for i in cols:
    #fit model to each variable
    norm_regr.fit(norm_variables20[[i]], target20)
    #add model results to list
    rows.append([i, norm_regr.intercept_, norm_regr.coef_, 
                 norm_regr.score(norm_variables20[[i]], target20)])

#turn list into dataframe
norm_linears20 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
norm_linears20['coefficient'] = norm_linears20['coefficient'].str.get(0)
norm_linears20['coefficient'] = norm_linears20['coefficient'].str.get(0)
norm_linears20['intercept'] = norm_linears20['intercept'].str.get(0)

#print df ordered by largest to smallest coef
norm_linears20.sort_values(by='coefficient', ascending=False)

#### 2010 Regressions

In [ ]:
### 2010 multi regression

#put all variables for predicting 2010 rates in dataframe
variables10 = df[['total_population', 
 'white_alone', 'black_alone', 'amerindian_alone', 'asian_alone', 
 'pacific_islander_alone',
 'other_alone', 'two_or_more', 'two_more_including_other',
 'two_more_excluding_other', 'total_education', 'no_school',
 'some_school', 'diploma', 'ged', 'some_college', 'associate',
 'bachelor', 'master', 'prof_school', 'doctorate', 'language_total',
 'lang_english_only', 'lang_spanish', 'lang_spanish_limited_english', 
 'lang_other_indo_euro', 'lang_asian_pacific_island',
 'lang_other', 'income_poverty_ratio', 'income_poverty_under_half',
 'income_poverty_half_.99', 'income_povery_1_1.24',
 'income_poverty_1.25_1.49', 'income_poverty_1.5_1.84',
 'income_poverty_1.85_1.99', 'income_poverty_2_over',
 'median_household_income', 'per_capita_income',
 'employment_total', 'labor_force', 'civilian_labor_force',
 'civilian_employed', 'civilian_unemployed', 'armed_forces',
 'not_labor_force', 'total_houses', 'occupied_houses',
 'vacant_houses', 'total_occupied_houses', 'owner_occupied',
 'renter_occupied', 'median_gross_rent', 'rent_to_income',
 'rent_less_10', 'rent_10_14.9', 'rent_15_19.9', 'rent_20_24.9',
 'rent_25_29.9', 'rent_30_34.9', 'rent_35_39.9', 'rent_40_49.9',
 'rent_50_over', 'rent_30_more', 'rent_not_computed', 'total_computer_status',
 'has_computer', 'dial_up_computer', 'broadband_computer',
 'no_internet_computer', 'no_computer', 'us_pop',
 'us_born', 'us_territory_born', 'us_born_abroad',
 'us_naturalization', 'not_us_citizen']]

#what we want to predict - 2010 response rates - in separate dataframe
target10 = df[["2010_tract_rate"]]

#create model and print summary table
model10 = sm.OLS(target10, variables10).fit()
print(model10.summary())

In [ ]:
### 2010 linear regression for each variable

#list of variable names
cols = variables10.columns.tolist()
#build multi-reg model
regr = linear_model.LinearRegression()
#create empty list for loop results
rows = []

#loop through variables
for i in cols:
    #fit a model to the current variable
    regr.fit(variables10[[i]], target10)
    #save the model's resulting variable name, intercept, coef, and r^2
    rows.append([i, regr.intercept_, regr.coef_,
                regr.score(variables10[[i]], target10)])

#turn list into data frame
linears10 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
linears10['coefficient'] = linears10['coefficient'].str.get(0)
linears10['coefficient'] = linears10['coefficient'].str.get(0)
linears10['intercept'] = linears10['intercept'].str.get(0)

#print data frame ordered coefficient largest --> smallest
linears10.sort_values(by='coefficient', ascending=False)

#### Normalized 2010 Regressions

In [ ]:
### 2010 normalized multi-regression

#normalize the variables
norm_variables10 = (variables10 - variables10.min()) / (variables10.max() - variables10.min())

#build normalized model and print summary
norm_model10 = sm.OLS(target10, norm_variables10).fit()
print(norm_model10.summary())

In [ ]:
###2010 normalized linear regressions for each variable

#create list of variable names
cols = norm_variables10.columns.tolist()
#build the model
norm_regr = linear_model.LinearRegression()
#create empty list for model results
rows = []
#cycle through variables
for i in cols:
    #do the linear regression on the current variable
    norm_regr.fit(norm_variables10[[i]], target10)
    #add the corresponding variable name, intercept, coefficient and r-squared to the list
    rows.append([i, norm_regr.intercept_, norm_regr.coef_,
                norm_regr.score(norm_variables10[[i]], target10)])

#turn list into data frame with these column names
norm_linears10 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
norm_linears10['coefficient'] = norm_linears10['coefficient'].str.get(0)
norm_linears10['coefficient'] = norm_linears10['coefficient'].str.get(0)
norm_linears10['intercept'] = norm_linears10['intercept'].str.get(0)

#print df sorted by coefficient
norm_linears10.sort_values(by='coefficient', ascending=False)

#### 2020 Edited Regressions
Edited = Run regressions with fewer variables

In [ ]:
### 2020 edited multi-regressions

#put all variables for predicting 2020 rates in dataframe
variables_ed = df[['2010_tract_rate','total_population', 
 'white_alone', 'black_alone', 'amerindian_alone', 'asian_alone', 
 'pacific_islander_alone', 'no_school',
 'some_school', 'diploma', 'ged', 'some_college', 'associate',
 'bachelor', 'master',
 'lang_english_only', 'lang_spanish', 'lang_spanish_limited_english', 
 'lang_other', 'income_poverty_ratio', 'per_capita_income',
 'civilian_employed', 'civilian_unemployed',
 'not_labor_force', 'total_houses', 'occupied_houses',
 'vacant_houses', 'owner_occupied',
 'renter_occupied', 'median_gross_rent', 'rent_to_income','rent_30_more',
 'has_computer', 'dial_up_computer', 'broadband_computer',
 'no_internet_computer', 'no_computer',
 'us_born', 'us_naturalization', 'not_us_citizen']]

#what we want to predict - 2020 response rates - in dataframe
target_ed = df[["2020_tract_rate"]]

#build and fit the multi-regression model
model_ed = sm.OLS(target_ed, variables_ed).fit()
#print out the model summary table
print(model_ed.summary())

In [ ]:
### 2020 edited linear regressions

#create list of variable names
cols = variables_ed.columns.tolist()
#build the model
regr = linear_model.LinearRegression()
#create empty list to append results
rows = []

#loop through variables
for i in cols:
    #fit the model
    regr.fit(variables_ed[[i]], target_ed)
    #put model variable name, intercept, coef and r^2 in list
    rows.append([i, regr.intercept_, regr.coef_,
                regr.score(variables_ed[[i]], target_ed)])

#turn list into data frame with these column names
linears_ed = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
linears_ed['coefficient'] = linears_ed['coefficient'].str.get(0)
linears_ed['coefficient'] = linears_ed['coefficient'].str.get(0)
linears_ed['intercept'] = linears_ed['intercept'].str.get(0)

#print df sorted by coefficient largest --> smallest
linears_ed.sort_values(by='coefficient', ascending=False)

In [ ]:
### normalized 2020 edited variables multi regression

#normalize the variables
norm_variables_ed = (variables_ed - variables_ed.min()) / (variables_ed.max() - variables_ed.min())

#build normalized model and print summary
norm_model_ed = sm.OLS(target_ed, norm_variables_ed).fit()
print(norm_model_ed.summary())

### Without Puerto Rico

#### 2020 regressions

In [ ]:
### 2020 Multi-regression

#df without puerto rico for regression 
no_pr = df[df['Region'] != 'Puerto Rico']
#this is different from earlier code no_pr = states[states.State != 'Puerto Rico']
#earlier code omitted Puerto Rico from state level data. this eliminates from tract level data


import statsmodels.api as sm

#put all variables for predicting 2020 rates in dataframe
variables20 = no_pr[['2010_tract_rate','total_population', 
 'white_alone', 'black_alone', 'amerindian_alone', 'asian_alone', 
 'pacific_islander_alone',
 'other_alone', 'two_or_more', 'two_more_including_other',
 'two_more_excluding_other', 'total_education', 'no_school',
 'some_school', 'diploma', 'ged', 'some_college', 'associate',
 'bachelor', 'master', 'prof_school', 'doctorate', 'language_total',
 'lang_english_only', 'lang_spanish', 'lang_spanish_limited_english', 
 'lang_other_indo_euro', 'lang_asian_pacific_island',
 'lang_other', 'income_poverty_ratio', 'income_poverty_under_half',
 'income_poverty_half_.99', 'income_povery_1_1.24',
 'income_poverty_1.25_1.49', 'income_poverty_1.5_1.84',
 'income_poverty_1.85_1.99', 'income_poverty_2_over',
 'median_household_income', 'per_capita_income',
 'employment_total', 'labor_force', 'civilian_labor_force',
 'civilian_employed', 'civilian_unemployed', 'armed_forces',
 'not_labor_force', 'total_houses', 'occupied_houses',
 'vacant_houses', 'total_occupied_houses', 'owner_occupied',
 'renter_occupied', 'median_gross_rent', 'rent_to_income',
 'rent_less_10', 'rent_10_14.9', 'rent_15_19.9', 'rent_20_24.9',
 'rent_25_29.9', 'rent_30_34.9', 'rent_35_39.9', 'rent_40_49.9',
 'rent_50_over', 'rent_30_more', 'rent_not_computed', 'total_computer_status',
 'has_computer', 'dial_up_computer', 'broadband_computer',
 'no_internet_computer', 'no_computer', 'us_pop',
 'us_born', 'us_territory_born', 'us_born_abroad',
 'us_naturalization', 'not_us_citizen']]

#what we want to predict - 2020 response rates - in dataframe
target20 = no_pr[["2020_tract_rate"]]

#build model and print summary
model20 = sm.OLS(target20, variables20).fit()
model20.summary()

In [ ]:
### 2020 linear regressions for each variable

from sklearn import linear_model

#create list of variable names
cols = variables20.columns.tolist()
#build linear model
regr = linear_model.LinearRegression()
#create empty list to store loop results
rows = []
#loop through each variable
for i in cols:
    #fit linear model to variable
    regr.fit(variables20[[i]], target20)
    #save model variable name, intercept, coef and r^2 to list
    rows.append([i, regr.intercept_, regr.coef_, regr.score(variables20[[i]], target20)])

#turn list into df with these column names
linears20 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets lol
linears20['coefficient'] = linears20['coefficient'].str.get(0)
linears20['coefficient'] = linears20['coefficient'].str.get(0)
linears20['intercept'] = linears20['intercept'].str.get(0)

#print df sorted coefs largest --> smallest
linears20.sort_values(by='coefficient', ascending=False)

#### Normalized 2020 inputs

In [ ]:
### 2020 Multi regression with normalized variables

#normalize variable values
norm_variables20 = (variables20 - variables20.min()) / (variables20.max() - variables20.min())

#build normalized multi-regress model and print summary
norm_model20 = sm.OLS(target20, norm_variables20).fit()
norm_model20.summary()

In [ ]:
#2020 normalized linear regressions for each variable

# linear regression for each variable 'i'
cols = norm_variables20.columns.tolist()
# create model
norm_regr = linear_model.LinearRegression()
#empty list for loop results
rows = []
#loop through each variable
for i in cols:
    #fit model to each variable
    norm_regr.fit(norm_variables20[[i]], target20)
    #add model results to list
    rows.append([i, norm_regr.intercept_, norm_regr.coef_, 
                 norm_regr.score(norm_variables20[[i]], target20)])

#turn list into dataframe
norm_linears20 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
norm_linears20['coefficient'] = norm_linears20['coefficient'].str.get(0)
norm_linears20['coefficient'] = norm_linears20['coefficient'].str.get(0)
norm_linears20['intercept'] = norm_linears20['intercept'].str.get(0)

#print df ordered by largest to smallest coef
norm_linears20.sort_values(by='coefficient', ascending=False)

#### 2010 Regressions

In [ ]:
### 2010 multi regression

#put all variables for predicting 2010 rates in dataframe
variables10 = no_pr[['total_population', 
 'white_alone', 'black_alone', 'amerindian_alone', 'asian_alone', 
 'pacific_islander_alone',
 'other_alone', 'two_or_more', 'two_more_including_other',
 'two_more_excluding_other', 'total_education', 'no_school',
 'some_school', 'diploma', 'ged', 'some_college', 'associate',
 'bachelor', 'master', 'prof_school', 'doctorate', 'language_total',
 'lang_english_only', 'lang_spanish', 'lang_spanish_limited_english', 
 'lang_other_indo_euro', 'lang_asian_pacific_island',
 'lang_other', 'income_poverty_ratio', 'income_poverty_under_half',
 'income_poverty_half_.99', 'income_povery_1_1.24',
 'income_poverty_1.25_1.49', 'income_poverty_1.5_1.84',
 'income_poverty_1.85_1.99', 'income_poverty_2_over',
 'median_household_income', 'per_capita_income',
 'employment_total', 'labor_force', 'civilian_labor_force',
 'civilian_employed', 'civilian_unemployed', 'armed_forces',
 'not_labor_force', 'total_houses', 'occupied_houses',
 'vacant_houses', 'total_occupied_houses', 'owner_occupied',
 'renter_occupied', 'median_gross_rent', 'rent_to_income',
 'rent_less_10', 'rent_10_14.9', 'rent_15_19.9', 'rent_20_24.9',
 'rent_25_29.9', 'rent_30_34.9', 'rent_35_39.9', 'rent_40_49.9',
 'rent_50_over', 'rent_30_more', 'rent_not_computed', 'total_computer_status',
 'has_computer', 'dial_up_computer', 'broadband_computer',
 'no_internet_computer', 'no_computer', 'us_pop',
 'us_born', 'us_territory_born', 'us_born_abroad',
 'us_naturalization', 'not_us_citizen']]

#what we want to predict - 2010 response rates - in separate dataframe
target10 = no_pr[["2010_tract_rate"]]

#create model and print summary table
model10 = sm.OLS(target10, variables10).fit()
model10.summary()

In [ ]:
### 2010 linear regression for each variable

#list of variable names
cols = variables10.columns.tolist()
#build multi-reg model
regr = linear_model.LinearRegression()
#create empty list for loop results
rows = []

#loop through variables
for i in cols:
    #fit a model to the current variable
    regr.fit(variables10[[i]], target10)
    #save the model's resulting variable name, intercept, coef, and r^2
    rows.append([i, regr.intercept_, regr.coef_,
                regr.score(variables10[[i]], target10)])

#turn list into data frame
linears10 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
linears10['coefficient'] = linears10['coefficient'].str.get(0)
linears10['coefficient'] = linears10['coefficient'].str.get(0)
linears10['intercept'] = linears10['intercept'].str.get(0)

#print data frame ordered coefficient largest --> smallest
linears10.sort_values(by='coefficient', ascending=False)

#### Normalized 2010 Regressions

In [ ]:
### 2010 normalized multi-regression

#normalize the variables
norm_variables10 = (variables10 - variables10.min()) / (variables10.max() - variables10.min())

#build normalized model and print summary
norm_model10 = sm.OLS(target10, norm_variables10).fit()
norm_model10.summary()

In [ ]:
###2010 normalized linear regressions for each variable

#create list of variable names
cols = norm_variables10.columns.tolist()
#build the model
norm_regr = linear_model.LinearRegression()
#create empty list for model results
rows = []
#cycle through variables
for i in cols:
    #do the linear regression on the current variable
    norm_regr.fit(norm_variables10[[i]], target10)
    #add the corresponding variable name, intercept, coefficient and r-squared to the list
    rows.append([i, norm_regr.intercept_, norm_regr.coef_,
                norm_regr.score(norm_variables10[[i]], target10)])

#turn list into data frame with these column names
norm_linears10 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
norm_linears10['coefficient'] = norm_linears10['coefficient'].str.get(0)
norm_linears10['coefficient'] = norm_linears10['coefficient'].str.get(0)
norm_linears10['intercept'] = norm_linears10['intercept'].str.get(0)

#print df sorted by coefficient
norm_linears10.sort_values(by='coefficient', ascending=False)

#### 2020 Edited Regressions
Edited = Run regressions with fewer variables

In [ ]:
### 2020 edited multi-regressions

#put all variables for predicting 2020 rates in dataframe
variables_ed = no_pr[['2010_tract_rate','total_population', 
 'white_alone', 'black_alone', 'amerindian_alone', 'asian_alone', 
 'pacific_islander_alone', 'no_school',
 'some_school', 'diploma', 'ged', 'some_college', 'associate',
 'bachelor', 'master',
 'lang_english_only', 'lang_spanish', 'lang_spanish_limited_english', 
 'lang_other', 'income_poverty_ratio', 'per_capita_income',
 'civilian_employed', 'civilian_unemployed',
 'not_labor_force', 'total_houses', 'occupied_houses',
 'vacant_houses', 'owner_occupied',
 'renter_occupied', 'median_gross_rent', 'rent_to_income','rent_30_more',
 'has_computer', 'dial_up_computer', 'broadband_computer',
 'no_internet_computer', 'no_computer',
 'us_born', 'us_naturalization', 'not_us_citizen']]

#what we want to predict - 2020 response rates - in dataframe
target_ed = no_pr[["2020_tract_rate"]]

#build and fit the multi-regression model
model_ed = sm.OLS(target_ed, variables_ed).fit()
#print out the model summary table
model_ed.summary()

In [ ]:
### 2020 edited linear regressions

#create list of variable names
cols = variables_ed.columns.tolist()
#build the model
regr = linear_model.LinearRegression()
#create empty list to append results
rows = []

#loop through variables
for i in cols:
    #fit the model
    regr.fit(variables_ed[[i]], target_ed)
    #put model variable name, intercept, coef and r^2 in list
    rows.append([i, regr.intercept_, regr.coef_,
                regr.score(variables_ed[[i]], target_ed)])

#turn list into data frame with these column names
linears_ed = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
linears_ed['coefficient'] = linears_ed['coefficient'].str.get(0)
linears_ed['coefficient'] = linears_ed['coefficient'].str.get(0)
linears_ed['intercept'] = linears_ed['intercept'].str.get(0)

#print df sorted by coefficient largest --> smallest
linears_ed.sort_values(by='coefficient', ascending=False)

In [ ]:
### normalized 2020 edited variables multi regression

#normalize the variables
norm_variables_ed = (variables_ed - variables_ed.min()) / (variables_ed.max() - variables_ed.min())

#build normalized model and print summary
norm_model_ed = sm.OLS(target_ed, norm_variables_ed).fit()
norm_model_ed.summary()

### Puerto Rico

In [ ]:
#create df with just puerto rico
is_pr = df['Region'] == 'Puerto Rico'
pr = df[is_pr]
pr

#### 2020 regressions

In [ ]:
### 2020 Multi-regression

import statsmodels.api as sm

#put all variables for predicting 2020 rates in dataframe
variables20 = pr[['2010_tract_rate','total_population', 
 'white_alone', 'black_alone', 'amerindian_alone', 'asian_alone', 
 'pacific_islander_alone',
 'other_alone', 'two_or_more', 'two_more_including_other',
 'two_more_excluding_other', 'total_education', 'no_school',
 'some_school', 'diploma', 'ged', 'some_college', 'associate',
 'bachelor', 'master', 'prof_school', 'doctorate', 'language_total',
 'lang_english_only', 'lang_spanish', 'lang_spanish_limited_english', 
 'lang_other_indo_euro', 'lang_asian_pacific_island',
 'lang_other', 'income_poverty_ratio', 'income_poverty_under_half',
 'income_poverty_half_.99', 'income_povery_1_1.24',
 'income_poverty_1.25_1.49', 'income_poverty_1.5_1.84',
 'income_poverty_1.85_1.99', 'income_poverty_2_over',
 'median_household_income', 'per_capita_income',
 'employment_total', 'labor_force', 'civilian_labor_force',
 'civilian_employed', 'civilian_unemployed', 'armed_forces',
 'not_labor_force', 'total_houses', 'occupied_houses',
 'vacant_houses', 'total_occupied_houses', 'owner_occupied',
 'renter_occupied', 'median_gross_rent', 'rent_to_income',
 'rent_less_10', 'rent_10_14.9', 'rent_15_19.9', 'rent_20_24.9',
 'rent_25_29.9', 'rent_30_34.9', 'rent_35_39.9', 'rent_40_49.9',
 'rent_50_over', 'rent_30_more', 'rent_not_computed', 'total_computer_status',
 'has_computer', 'dial_up_computer', 'broadband_computer',
 'no_internet_computer', 'no_computer', 'us_pop',
 'us_born', 'us_territory_born', 'us_born_abroad',
 'us_naturalization', 'not_us_citizen']]

#what we want to predict - 2020 response rates - in dataframe
target20 = pr[["2020_tract_rate"]]

#build model and print summary
model20 = sm.OLS(target20, variables20).fit()
model20.summary()

In [ ]:
### 2020 linear regressions for each variable

from sklearn import linear_model

#create list of variable names
cols = variables20.columns.tolist()
#build linear model
regr = linear_model.LinearRegression()
#create empty list to store loop results
rows = []
#loop through each variable
for i in cols:
    #fit linear model to variable
    regr.fit(variables20[[i]], target20)
    #save model variable name, intercept, coef and r^2 to list
    rows.append([i, regr.intercept_, regr.coef_, regr.score(variables20[[i]], target20)])

#turn list into df with these column names
linears20 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets lol
linears20['coefficient'] = linears20['coefficient'].str.get(0)
linears20['coefficient'] = linears20['coefficient'].str.get(0)
linears20['intercept'] = linears20['intercept'].str.get(0)

#print df sorted coefs largest --> smallest
linears20.sort_values(by='coefficient', ascending=False)

#### Normalized 2020 inputs

In [ ]:
### 2020 Multi regression with normalized variables

#normalize variable values
norm_variables20 = (variables20 - variables20.min()) / (variables20.max() - variables20.min())

#build normalized multi-regress model and print summary
norm_model20 = sm.OLS(target20, norm_variables20).fit()
norm_model20.summary()

In [ ]:
#2020 normalized linear regressions for each variable

# linear regression for each variable 'i'
cols = norm_variables20.columns.tolist()
# create model
norm_regr = linear_model.LinearRegression()
#empty list for loop results
rows = []
#loop through each variable
for i in cols:
    #fit model to each variable
    norm_regr.fit(norm_variables20[[i]], target20)
    #add model results to list
    rows.append([i, norm_regr.intercept_, norm_regr.coef_, 
                 norm_regr.score(norm_variables20[[i]], target20)])

#turn list into dataframe
norm_linears20 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
norm_linears20['coefficient'] = norm_linears20['coefficient'].str.get(0)
norm_linears20['coefficient'] = norm_linears20['coefficient'].str.get(0)
norm_linears20['intercept'] = norm_linears20['intercept'].str.get(0)

#print df ordered by largest to smallest coef
norm_linears20.sort_values(by='coefficient', ascending=False)

#### 2010 Regressions

In [ ]:
### 2010 multi regression

#put all variables for predicting 2010 rates in dataframe
variables10 = pr[['total_population', 
 'white_alone', 'black_alone', 'amerindian_alone', 'asian_alone', 
 'pacific_islander_alone',
 'other_alone', 'two_or_more', 'two_more_including_other',
 'two_more_excluding_other', 'total_education', 'no_school',
 'some_school', 'diploma', 'ged', 'some_college', 'associate',
 'bachelor', 'master', 'prof_school', 'doctorate', 'language_total',
 'lang_english_only', 'lang_spanish', 'lang_spanish_limited_english', 
 'lang_other_indo_euro', 'lang_asian_pacific_island',
 'lang_other', 'income_poverty_ratio', 'income_poverty_under_half',
 'income_poverty_half_.99', 'income_povery_1_1.24',
 'income_poverty_1.25_1.49', 'income_poverty_1.5_1.84',
 'income_poverty_1.85_1.99', 'income_poverty_2_over',
 'median_household_income', 'per_capita_income',
 'employment_total', 'labor_force', 'civilian_labor_force',
 'civilian_employed', 'civilian_unemployed', 'armed_forces',
 'not_labor_force', 'total_houses', 'occupied_houses',
 'vacant_houses', 'total_occupied_houses', 'owner_occupied',
 'renter_occupied', 'median_gross_rent', 'rent_to_income',
 'rent_less_10', 'rent_10_14.9', 'rent_15_19.9', 'rent_20_24.9',
 'rent_25_29.9', 'rent_30_34.9', 'rent_35_39.9', 'rent_40_49.9',
 'rent_50_over', 'rent_30_more', 'rent_not_computed', 'total_computer_status',
 'has_computer', 'dial_up_computer', 'broadband_computer',
 'no_internet_computer', 'no_computer', 'us_pop',
 'us_born', 'us_territory_born', 'us_born_abroad',
 'us_naturalization', 'not_us_citizen']]

#what we want to predict - 2010 response rates - in separate dataframe
target10 = pr[["2010_tract_rate"]]

#create model and print summary table
model10 = sm.OLS(target10, variables10).fit()
model10.summary()

In [ ]:
### 2010 linear regression for each variable

#list of variable names
cols = variables10.columns.tolist()
#build multi-reg model
regr = linear_model.LinearRegression()
#create empty list for loop results
rows = []

#loop through variables
for i in cols:
    #fit a model to the current variable
    regr.fit(variables10[[i]], target10)
    #save the model's resulting variable name, intercept, coef, and r^2
    rows.append([i, regr.intercept_, regr.coef_,
                regr.score(variables10[[i]], target10)])

#turn list into data frame
linears10 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
linears10['coefficient'] = linears10['coefficient'].str.get(0)
linears10['coefficient'] = linears10['coefficient'].str.get(0)
linears10['intercept'] = linears10['intercept'].str.get(0)

#print data frame ordered coefficient largest --> smallest
linears10.sort_values(by='coefficient', ascending=False)

#### Normalized 2010 Regressions

In [ ]:
### 2010 normalized multi-regression

#normalize the variables
norm_variables10 = (variables10 - variables10.min()) / (variables10.max() - variables10.min())

#build normalized model and print summary
norm_model10 = sm.OLS(target10, norm_variables10).fit()
norm_model10.summary()

In [ ]:
###2010 normalized linear regressions for each variable

#create list of variable names
cols = norm_variables10.columns.tolist()
#build the model
norm_regr = linear_model.LinearRegression()
#create empty list for model results
rows = []
#cycle through variables
for i in cols:
    #do the linear regression on the current variable
    norm_regr.fit(norm_variables10[[i]], target10)
    #add the corresponding variable name, intercept, coefficient and r-squared to the list
    rows.append([i, norm_regr.intercept_, norm_regr.coef_,
                norm_regr.score(norm_variables10[[i]], target10)])

#turn list into data frame with these column names
norm_linears10 = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
norm_linears10['coefficient'] = norm_linears10['coefficient'].str.get(0)
norm_linears10['coefficient'] = norm_linears10['coefficient'].str.get(0)
norm_linears10['intercept'] = norm_linears10['intercept'].str.get(0)

#print df sorted by coefficient
norm_linears10.sort_values(by='coefficient', ascending=False)

#### 2020 Edited Regressions
Edited = Run regressions with fewer variables

In [ ]:
### 2020 edited multi-regressions

#put all variables for predicting 2020 rates in dataframe
variables_ed = pr[['2010_tract_rate','total_population', 
 'white_alone', 'black_alone', 'amerindian_alone', 'asian_alone', 
 'pacific_islander_alone', 'no_school',
 'some_school', 'diploma', 'ged', 'some_college', 'associate',
 'bachelor', 'master',
 'lang_english_only', 'lang_spanish', 'lang_spanish_limited_english', 
 'lang_other', 'income_poverty_ratio', 'per_capita_income',
 'civilian_employed', 'civilian_unemployed',
 'not_labor_force', 'total_houses', 'occupied_houses',
 'vacant_houses', 'owner_occupied',
 'renter_occupied', 'median_gross_rent', 'rent_to_income','rent_30_more',
 'has_computer', 'dial_up_computer', 'broadband_computer',
 'no_internet_computer', 'no_computer',
 'us_born', 'us_naturalization', 'not_us_citizen']]

#what we want to predict - 2020 response rates - in dataframe
target_ed = pr[["2020_tract_rate"]]

#build and fit the multi-regression model
model_ed = sm.OLS(target_ed, variables_ed).fit()
#print out the model summary table
model_ed.summary()

In [ ]:
### 2020 edited linear regressions

#create list of variable names
cols = variables_ed.columns.tolist()
#build the model
regr = linear_model.LinearRegression()
#create empty list to append results
rows = []

#loop through variables
for i in cols:
    #fit the model
    regr.fit(variables_ed[[i]], target_ed)
    #put model variable name, intercept, coef and r^2 in list
    rows.append([i, regr.intercept_, regr.coef_,
                regr.score(variables_ed[[i]], target_ed)])

#turn list into data frame with these column names
linears_ed = pd.DataFrame(rows, columns=['variable', 'intercept', 'coefficient', 'r-squared'])

#remove square brackets
linears_ed['coefficient'] = linears_ed['coefficient'].str.get(0)
linears_ed['coefficient'] = linears_ed['coefficient'].str.get(0)
linears_ed['intercept'] = linears_ed['intercept'].str.get(0)

#print df sorted by coefficient largest --> smallest
linears_ed.sort_values(by='coefficient', ascending=False)

In [ ]:
### normalized 2020 edited variables multi regression

#normalize the variables
norm_variables_ed = (variables_ed - variables_ed.min()) / (variables_ed.max() - variables_ed.min())

#build normalized model and print summary
norm_model_ed = sm.OLS(target_ed, norm_variables_ed).fit()
norm_model_ed.summary()